In [8]:
!launch_scripts/baseline.sh "photo,cartoon,sketch"

100%|███████████████████████████████████████████| 31/31 [00:02<00:00, 12.89it/s]


In [ ]:
import os
def check_if_experiment_already_run(ALPHA, APPLY_EVERY_N, SKIP_FIRST_N, EXP_TARGET):
  LOG_FILENAME = f"SKIP_{SKIP_FIRST_N}_ALPHA_{ALPHA}-APPLY_EVERY_N_{APPLY_EVERY_N}-log.txt"
  LOG_PATH = os.path.join(f"record/activation_shaping_experiments/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, alphas, apply_every_n_list, skip_first_n_list):
  for alpha in alphas:


    for apply_every_n in apply_every_n_list:
      for skip_first_n in skip_first_n_list:
        if (not check_if_experiment_already_run(alpha, apply_every_n, skip_first_n, category)):
          !launch_scripts/activation_shaping_experiments.sh {category} "" "{alpha}" "{apply_every_n}" "{skip_first_n}"
        else:
          print("Already existing")

          

alphas = [0.9, 0.7, 0.5]
apply_every_n_list = [0, 2]
skip_first_n_list = [15, 14, 13, 10, 6 ]
run_experiment("photo,cartoon,sketch", alphas, apply_every_n_list, skip_first_n_list)

In [ ]:
#import plotly.graph_objects as go
import numpy as np
import re

#Parse outputs from previous step and produce some statistics
def get_accuracy(EXP_TARGET, ALPHA, APPLY_EVERY_N, SKIP_FIRST_N):
  def get_max_accuracy_from_file(file_path):
    max_a_binarization = {}
    max_a_no_binarization = {}
    with open(file_path) as file:
      lines = file.readlines()
      for line in lines:
        pattern = r'TEST WITH BINARIZATION Evaluate: (\w+): Accuracy: (\d+\.\d+) - Loss: (\d+\.\d+)'
        match = re.match(pattern, line)
        if match:
            category = match.group(1)
            accuracy = float(match.group(2))
            if (category in max_a_binarization):
              max_a_binarization[category] = max(accuracy, max_a_binarization[category])
            else:
              max_a_binarization[category] = accuracy
        else:
            pattern = r'TEST WITHOUT BINARIZATION Evaluate: (\w+): Accuracy: (\d+\.\d+) - Loss: (\d+\.\d+)'
            match = re.match(pattern, line)
            if match:
                category = match.group(1)
                accuracy = float(match.group(2))
                if (category in max_a_no_binarization):
                    max_a_no_binarization[category] = max(accuracy, max_a_no_binarization[category])
                else:
                    max_a_no_binarization[category] = accuracy
    return max_a_binarization, max_a_no_binarization
  LOG_FILENAME = f"SKIP_{SKIP_FIRST_N}_ALPHA_{ALPHA}-APPLY_EVERY_N_{APPLY_EVERY_N}-log.txt"
  LOG_PATH = os.path.join(f"record/activation_shaping_experiments/{EXP_TARGET}", LOG_FILENAME)
  return get_max_accuracy_from_file(LOG_PATH)


class Results:
  def __init__(self, target):
    self.results_bin = {}
    self.results_no_bin = {}
    self.target = target

  def add_result(self, alpha, apply_every, skip_first):
    accuracy_bin, accuracy_no_bin = get_accuracy(self.target, alpha, apply_every, skip_first)
    
    for category in accuracy_bin.keys():
      if not category in self.results_bin:
        self.results_bin[category] = []
      self.results_bin[category].append({"accuracy": accuracy_bin[category], "alpha": alpha, "apply_every":apply_every, "skip_first": skip_first})
    for category in accuracy_no_bin.keys():
      if not category in self.results_no_bin:
        self.results_no_bin[category] = []
      self.results_no_bin[category].append({"accuracy": accuracy_no_bin[category], "alpha": alpha, "apply_every":apply_every, "skip_first": skip_first})

  def get_results_bin(self, category, alpha=None, apply_every=None, skip_first=None):
    to_return = []
    for result in self.results_bin[category]:
      if ((alpha is None or result["alpha"] == alpha)
      and (apply_every is None or result["apply_every"] == apply_every)
      and (skip_first is None or result["skip_first"] == skip_first)):
        to_return.append(result)
    return to_return
  def get_results_no_bin(self, category, alpha=None, apply_every=None, skip_first=None):
    to_return = []
    for result in self.results_no_bin[category]:
      if ((alpha is None or result["alpha"] == alpha)
      and (apply_every is None or result["apply_every"] == apply_every)
      and (skip_first is None or result["skip_first"] == skip_first)):
        to_return.append(result)
    return to_return


alphas = [0.9, 0.7, 0.5]
apply_every_n_list = [0, 2]
skip_first_n_list = [15, 14, 13, 10, 6 ]
categories = ["photo","cartoon","sketch"]
results = Results("photo,cartoon,sketch")
for alpha in alphas:
  for apply_every in apply_every_n_list:
    for skip_first in skip_first_n_list:
      results.add_result(alpha, apply_every, skip_first)

"""def make_plot(plot_title, raw_results, axis_values):
  y_max =[max(result, key=lambda r: r["accuracy"]) for result in raw_results]
  y_mean = [sum(result, key=lambda r: r["accuracy"]) / len(result) for result in raw_results]
  # Create traces
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=axis_values, y=y_max,
                      mode='lines',
                      name=f"{plot_title} - mean"))
  fig.add_trace(go.Scatter(x=axis_values, y=y_mean,
                      mode='lines',
                      name=f"{plot_title} - avg"))
  fig.show()"""

#make_plot("By alpha", [results.get_results("photos", alpha=a) for a in alphas])

for category in categories:
    print(f"\nResults for {category} - with binarization in the test mode")
    #Get max accuracy
    max_accuracy = max(results.get_results_bin(category), key=lambda r: r["accuracy"])
    print(f"Max accuracy: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by alpha
    for a in alphas:
        max_accuracy = max(results.get_results_bin(category, alpha=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for a = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by apply_every
    for a in apply_every_n_list:
        max_accuracy = max(results.get_results_bin(category, apply_every=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for apply_every = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by skip_first_n_list
    for a in skip_first_n_list:
        max_accuracy = max(results.get_results_bin(category, skip_first=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for skip_first_n_list = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")

    print(f"\nResults for {category} - without binarization in the test mode")
    #Get max accuracy
    max_accuracy = max(results.get_results_no_bin(category), key=lambda r: r["accuracy"])
    print(f"Max accuracy: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by alpha
    for a in alphas:
        max_accuracy = max(results.get_results_no_bin(category, alpha=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for a = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by apply_every
    for a in apply_every_n_list:
        max_accuracy = max(results.get_results_no_bin(category, apply_every=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for apply_every = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")
    #Get max accuracy by skip_first_n_list
    for a in skip_first_n_list:
        max_accuracy = max(results.get_results_no_bin(category, skip_first=a), key=lambda r: r["accuracy"])
        print(f"Max accuracy for skip_first_n_list = {a}: {max_accuracy['accuracy']} - Params: {max_accuracy}")


In [1]:
#DOMAIN ADAPTATION STANDARD

import os
def check_if_experiment_already_run(K, APPLY_EVERY_N, SKIP_FIRST_N, EXP_TARGET, RECORD_MODE):
  LOG_FILENAME = f"SKIP_{SKIP_FIRST_N}_K_{K}-APPLY_EVERY_N_{APPLY_EVERY_N}-RECORD_MODE{RECORD_MODE}-log.txt"
  LOG_PATH = os.path.join(f"record/domain_adaptation/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, configurations):
  for configuration in configurations:
    apply_every_n = configuration["apply_every_n"]
    skip_first_n = configuration["skip_first_n"]
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    if (not check_if_experiment_already_run(K, apply_every_n, skip_first_n, category, record_mode)):
        !launch_scripts/domain_adaptation.sh {category} "" {K} {apply_every_n} {skip_first_n} {record_mode}
    else:
        print("Already existing")

          

#threshold
configurations = []
for k in [0, 0.2, 0.4]:
    configurations.append({"apply_every_n": 2, "skip_first_n": 14, "K": k, "RECORD_MODE": "threshold"})
    configurations.append({"apply_every_n": 0, "skip_first_n": 14, "K": k, "RECORD_MODE": "threshold"})
    configurations.append({"apply_every_n": 0, "skip_first_n": 15, "K": k, "RECORD_MODE": "threshold"})
for target in ["photo","cartoon","sketch"]:
  run_experiment(target, configurations)
"""BEST:
Cartoon: 58, with skip 14, K=0, apply every 2
Photo: 84, with skip 15, K=0.2
Sketch: 34, with skip 15, K=0.2
"""

100%|███████████████████████████████████████████| 14/14 [00:01<00:00, 11.74it/s]
free(): corrupted unsorted chunks
launch_scripts/domain_adaptation.sh: line 14: 24471 Aborted                 python3 main.py --experiment=domain_adaptation --experiment_name=domain_adaptation/${target_domain}/ --dataset_args="{'root': 'data/PACS', 'source_domain': 'art_painting', 'target_domain': '${target_domain}','SKIP_FIRST_N': ${SKIP_FIRST_N},'APPLY_EVERY_N': ${APPLY_EVERY_N}, 'K': ${K}, 'RECORD_MODE': '${RECORD_MODE}' }" --batch_size=128 --num_workers=5 --grad_accum_steps=1 ${test_only}
100%|███████████████████████████████████████████| 14/14 [00:01<00:00, 11.28it/s]


'BEST:\nCartoon: 58, with skip 14, K=0, apply every 2\nPhoto: 84, with skip 15, K=0.2\nSketch: 34, with skip 15, K=0.2\n'

In [ ]:

import os
def check_if_experiment_already_run(K, APPLY_EVERY_N, SKIP_FIRST_N, EXP_TARGET, RECORD_MODE):
  LOG_FILENAME = f"SKIP_{SKIP_FIRST_N}_K_{K}-APPLY_EVERY_N_{APPLY_EVERY_N}-RECORD_MODE{RECORD_MODE}-log.txt"
  LOG_PATH = os.path.join(f"record/domain_adaptation_alt/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, configurations):
  for configuration in configurations:
    apply_every_n = configuration["apply_every_n"]
    skip_first_n = configuration["skip_first_n"]
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    if (not check_if_experiment_already_run(K, apply_every_n, skip_first_n, category, record_mode)):
        !launch_scripts/domain_adaptation_alt.sh {category} "" {K} {apply_every_n} {skip_first_n} {record_mode}
    else:
        print("Already existing")

          

#threshold
configurations = []
for k in [0.1, 0.2, 0.5, 0.8]:
    configurations.append({"apply_every_n": 2, "skip_first_n": 14, "K": k, "RECORD_MODE": "topk"})
for target in ["photo", "sketch"]:
  run_experiment(target, configurations)

  #TODO: applicando a troppi layer non migliora, forse migliora rendendo k più selettivo
  #TODO se questo approccio funziona, prova anche con topK
  #Si può anche pensare di applicarlo a + layers

In [ ]:
import os
def check_if_experiment_already_run(K, LAYERS, EXP_TARGET, RECORD_MODE):
  LOG_FILENAME = f"L_{LAYERS}__K__{K}__RECORD_MODE{RECORD_MODE}-log.txt"
  LOG_PATH = os.path.join(f"record/domain_generalization/{EXP_TARGET}", LOG_FILENAME)
  return os.path.exists(LOG_PATH)

def run_experiment(category, configurations):
  for configuration in configurations:
    K = configuration["K"]
    record_mode = configuration["RECORD_MODE"]
    layers = configuration["LAYERS"]
    if (not check_if_experiment_already_run(K, layers, category, record_mode)):
        !launch_scripts/domain_generalization.sh {category} "" {K} {record_mode} {layers}
    else:
        print("Already existing")


c = [
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "1"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "0"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,3"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,3,4"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,5"},
   {"apply_every_n": 2, "K": 0, "RECORD_MODE": "threshold", "LAYERS": "2,11"}
]
run_experiment("cartoon", c)

#TODO:
#Try skip_first_n = 10 o 9 e lasciare N layers di seguito!
#Try diversi valori per lr, weight_decay 
#Una volta scelta la configurazione sperimenta anche con K

In [ ]:
#TODO: possibile che randomizzando le copie si ottenga un miglioramento?